In [24]:
import os
from torchtext.vocab import build_vocab_from_iterator
from nltk.tokenize import word_tokenize
import torch
import re
from torch import nn

In [2]:
#there are 6 input tokens 
inputs = torch.tensor([
    [0.43, 0.15, 0.89],
    [0.55, 0.87, 0.66], # compute relative to x2 (start from 1)
    [0.57, 0.85, 0.64],
    [0.22, 0.58, 0.33],
    [0.77, 0.25, 0.10],
    [0.05, 0.80, 0.55]
])

query = inputs[1] #x2
attn_score2 = torch.empty(inputs.shape[0])
print(attn_score2.shape)

for i, xi in enumerate(inputs):
    attn_score2[i] = torch.dot(xi, query)

def atten_score(query, inputs):
    for i, xi in enumerate(inputs):
        attn_score2[i] = torch.dot(xi, query)

print(attn_score2) #attention score for x2 using each input tokens, shape = (6)

torch.Size([6])
tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [3]:
# Normalize attention score to get weights alpha

attn_weights = torch.softmax(attn_score2, dim = 0)
print(attn_weights)

# compute context vector z(2)
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

print('input shape = ', inputs.shape)
print('attn weights = ', attn_weights.shape)
print('z(2) = ', context_vec_2.shape)

for i, xi in enumerate(inputs):
    context_vec_2 += attn_weights[i] * xi

print(context_vec_2)



tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
input shape =  torch.Size([6, 3])
attn weights =  torch.Size([6])
z(2) =  torch.Size([3])
tensor([0.4419, 0.6515, 0.5683])


In [4]:
# Now compute attention scores for each input token (matrix of size 6x6)

attn_score = torch.empty(inputs.shape[0], inputs.shape[0])
for i, xi in enumerate(inputs):
    for j, xj in enumerate(inputs):
        attn_score[i,j] = torch.dot(xi, xj)
    
print('for loop mat = \n', attn_score)

#Achive the same with matrix multiplication
attn_score = inputs @ inputs.T #matmul of (6,3) @ (3,6) = (6,6)
print('matrix mat = \n', attn_score)

attn_weights = torch.softmax(attn_score, dim = -1)

print('attn weights \n', attn_weights)

#Compute the context variable 
# matmul atten weights and inputs, (6,6) @ (6,3) = (6,3)

context_vec = attn_weights @ inputs

print('context vec = \n', context_vec)


for loop mat = 
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
matrix mat = 
 tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
attn weights 
 tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958

# Attention with weights

In [5]:
#for illustration
x2 = inputs[1]
d_in = inputs.shape[1]
d_out =  2
print("d_in = ", d_in, " d_out = ", d_out)

W_query =   torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) # set False for illustration, set True to train later
W_key =     torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value =   torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

d_in =  3  d_out =  2


In [8]:
query2 = x2 @ W_query # if, x2 = (1,3), query2 = (3,2) -> (1,3) @ (3,2) = (1,2)
key2 = x2 @ W_key # if, x2 = (1,3), key2 = (3,2) -> (1,3) @ (3,2) = (1,2)
value2 = x2 @ W_value# if, x2 = (1,3), value2 = (3,2) -> (1,3) @ (3,2) = (1,2)

print('x2 = ', x2)
print(query2)
print(key2)
print(value2)

x2 =  tensor([0.5500, 0.8700, 0.6600])
tensor([1.5738, 0.7099])
tensor([1.6218, 1.1505])
tensor([1.1344, 0.9998])


In [9]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys shape = " , keys.shape)
print("values shape = " , values.shape)

keys shape =  torch.Size([6, 2])
values shape =  torch.Size([6, 2])


In [18]:
# Compute attention score with query, key, value weights for keys2
keys2 = keys[1]
print('keys 2 = ', keys2.shape)
print('query 2 = ', query2.shape)
attn_score_keys2 = query2.dot(keys2) #query shape = (2) @ keys2 = (2), product element-wise, produce 1 scalar
print('attn_score_keys2 = ', attn_score_keys2)

attn_scores2 = query2 @ keys.T # query shape = (2) @ (2, 6) = (1,6)
print("attn_scores2 = ", attn_scores2.shape)
# now generalize with all input tokens

keys 2 =  torch.Size([2])
query 2 =  torch.Size([2])
attn_score_keys2 =  tensor(3.3689)
attn_scores2 =  torch.Size([6])


In [20]:
# Now scale the attension score to get attention weights alpha 
d_k = keys.shape[-1]
print('attn score 2 shape = ', attn_score2.shape)
# The difference from earlier is that we now scale the attention scores 
# by dividing them by the square root of the embedding dimension of the keys
# this prevent small gradient updates, find out more about the math if interested
attn_weights2 = torch.softmax(attn_scores2 / d_k ** 0.5, dim = -1 ) #scale last dim
print("attn_weights2 = ", attn_weights2.shape)

attn score 2 shape =  torch.Size([6])
attn_weights2 =  torch.Size([6])


In [23]:
context_vec2 = attn_weights2 @ values #(1, 6) @ (6,2) = (1,2)
print('context_vec2.shape =', context_vec2.shape)
print(context_vec2)


context_vec2.shape = torch.Size([2])
tensor([0.9103, 0.8168])


# Illustration of Casual or Masked attention

In [47]:
# Mask with torch.tril

a = torch.rand(10,5)
a = torch.tril(a)

row_sums = a.sum(dim = 1, keepdim = True) # we need to normalize with sum
a = a / row_sums
print(a)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6748, 0.3252, 0.0000, 0.0000, 0.0000],
        [0.4921, 0.4694, 0.0385, 0.0000, 0.0000],
        [0.1040, 0.0967, 0.4190, 0.3803, 0.0000],
        [0.3708, 0.0487, 0.2435, 0.1567, 0.1802],
        [0.0731, 0.1301, 0.3603, 0.3073, 0.1293],
        [0.0308, 0.3394, 0.0509, 0.2695, 0.3093],
        [0.2580, 0.1070, 0.1628, 0.2173, 0.2549],
        [0.3150, 0.3167, 0.0844, 0.1255, 0.1584],
        [0.3131, 0.0236, 0.0093, 0.3060, 0.3480]])


In [48]:
# Applying -inf into upper triangular matrix, this will skip the sum normailzation part previous step
d = 5
a = torch.rand(d,d)
print(a)
mask = torch.triu(torch.ones(d,d), diagonal=1)

masked = a.masked_fill(mask.bool(), -torch.inf)
print(masked)

#apply softmax with dim norm
masked = torch.softmax(masked / keys.shape[1] ** 0.5, dim = 1)

print(masked)

tensor([[0.8058, 0.8493, 0.2060, 0.1110, 0.1066],
        [0.3080, 0.5040, 0.6367, 0.7604, 0.7225],
        [0.5267, 0.9733, 0.2127, 0.8008, 0.4209],
        [0.8832, 0.3727, 0.4386, 0.3557, 0.1285],
        [0.5635, 0.9688, 0.6450, 0.5883, 0.5512]])
tensor([[0.8058,   -inf,   -inf,   -inf,   -inf],
        [0.3080, 0.5040,   -inf,   -inf,   -inf],
        [0.5267, 0.9733, 0.2127,   -inf,   -inf],
        [0.8832, 0.3727, 0.4386, 0.3557,   -inf],
        [0.5635, 0.9688, 0.6450, 0.5883, 0.5512]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4654, 0.5346, 0.0000, 0.0000, 0.0000],
        [0.3152, 0.4323, 0.2525, 0.0000, 0.0000],
        [0.3209, 0.2237, 0.2344, 0.2210, 0.0000],
        [0.1852, 0.2466, 0.1962, 0.1884, 0.1836]])


# Using dropout in attention

In [50]:
# half of the elements in the matrix are randomly set to zero. 
# To compensate for the reduction in active elements, 
# the values of the remaining elements in the matrix are scaled up by a factor of 1/0.5 = 2
dropout = nn.Dropout(0.5)
a = torch.ones(5,5)
print(a)

print(dropout(a))

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 0., 2.],
        [2., 0., 0., 2., 0.],
        [2., 2., 0., 0., 2.],
        [0., 0., 0., 2., 2.],
        [2., 2., 0., 2., 2.]])


# Combine all that to final Python class

In [77]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout = 0.5, qkv_bias = False):
        super().__init__()
        #add linear weights
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)

        #add dropout
        self.dropout = nn.Dropout(dropout)

        #register to upper mask to buffer
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        batch, num_tokens, d_in = x.shape

        keys = self.W_key(x) #shape = (batch, num_tokens, d_out)
        values = self.W_value(x) #shape = (batch, num_tokens, d_out)
        queries = self.W_query(x) #shape = (batch, num_tokens, d_out)

        #get attention scores
        attn_scores: torch.Tensor = queries @ keys.transpose(1,2) #attn score shape = (batch, num_tokens, num_tokens)

        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], 
            -torch.inf
        )

        #get attention weights, shape = (batch, num_tokens, num_tokens)
        attn_weights = self.dropout( #apply dropout to weights
            torch.softmax(attn_scores / attn_scores.shape[-1] ** 0.5, dim = -1) #apply softmax to wieights
        )

        context_vec = attn_weights @ values # (b, n, n) @ (b, n, d_out) = (b, n, d_out)
        return context_vec

x = torch.rand(4, 5,10) 

attention = CasualAttention(10, 3, 5, 0.5)

compressed_vec = attention(x)

print('compressed vec = ', compressed_vec.shape)


compressed vec =  torch.Size([4, 5, 3])


In [68]:
mask = torch.triu(torch.ones(5,5), diagonal=1)
print(mask)

a = torch.rand(5,5)
a.masked_fill_(mask.bool(), -torch.inf)

print(a)

tensor([[0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.]])
tensor([[0.8112,   -inf,   -inf,   -inf,   -inf],
        [0.1152, 0.1078,   -inf,   -inf,   -inf],
        [0.4351, 0.4052, 0.0896,   -inf,   -inf],
        [0.7336, 0.7008, 0.0291, 0.4123,   -inf],
        [0.2637, 0.2708, 0.9994, 0.3198, 0.9544]])


In [54]:
a = torch.rand(5,2,3)
print(a.transpose(1,2).shape)

torch.Size([5, 3, 2])


# Multihead attention
“multi-head” refers to dividing the attention mechanism into multiple “heads,” each operating independently.

#### Sections
1. simple multi head by stacking up Casual Attention (reference this in the book)
2. multi head attention with weight split. More complex multi head but more efficient (code this part)

In [119]:
class MultiHeadAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, num_heads, dropout = 0.5, qkv_bias = False) -> None:
        super().__init__()
        assert d_out % num_heads == 0, "d out must be divisible by num_heads"

        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)

        self.d_in = d_in
        self.d_out = d_out
        self.head_dim = d_out // num_heads
        self.num_heads = num_heads

        self.out_proj = nn.Linear(d_out, d_out)

        nn.dropout = nn.Dropout(dropout)

        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
    
    def forward(self, x):

        batch, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        #split heads
        keys = keys.view(batch, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(batch, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batch, num_tokens, self.num_heads, self.head_dim)

        #transpose
        keys = keys.transpose(1,2) #shape = (batch, num_heads, num_tokens , head_dim)
        queries = queries.transpose(1,2) #shape = (batch, num_heads, num_tokens , head_dim)
        values = values.transpose(1,2) #shape = (batch, num_heads, num_tokens , head_dim)

        #do batch matmul
        #NOTE: can we improve this with torch.bmm ? 
        attn_scores:torch.Tensor = queries @ keys.transpose(2,3) # shape = (batch, num_heads , num_tokens, num_tokens)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1) # shape = (batch, num_heads, num_tokens, num_tokens)

        context_vec = (attn_weights @ values) #shape = (batch, num_heads, num_tokens, head_dim)
        context_vec = context_vec.transpose(1,2) #shape = (batch, num_tokens, num_heads, head_dim)

        context_vec = context_vec.contiguous().view(batch, num_tokens, self.d_out) 

        context_vec = self.out_proj(context_vec)

        return context_vec




In [120]:
batch = 4
num_tokens = 8
d_in = 3
d_out = 6
context_length = 10
num_heads = 2

x = torch.rand(batch, num_tokens, d_in)

attn = MultiHeadAttention(d_in, d_out, context_length, num_heads)

context = attn(x)

print('context = ' , context.shape)

context =  torch.Size([4, 8, 6])
